In [1]:
from depsurf import DBGSYM_PATH

import pickle

with open(DBGSYM_PATH / "subroutine_info.pkl", "rb") as f:
    subroutine_info = pickle.load(f)

In [2]:
flattened_info = [entry for v in subroutine_info.values() for entry in v.values()]

len(flattened_info)

79338

In [3]:
sum([len(d) for d in subroutine_info.values() if len(d) > 1]) / len(
    [v for v in flattened_info if not v.external]
)

0.02088613125928515

In [4]:
len([v for v in flattened_info if v.external]) / len(flattened_info)

0.2788449418941743

In [5]:
import pandas as pd


df = pd.DataFrame(
    [
        {
            "name": name,
            "location": location,
            "caller_inline": len(info.caller_inline),
            "caller_func": len(info.caller_func),
            "external": info.external,
        }
        for name, d in subroutine_info.items()
        for location, info in d.items()
    ]
)

print(len(df))

df[:10]

79338


,name,location,caller_inline,caller_func,external
0,kernel_init,init/main.c,0,0,False
1,mark_readonly,init/main.c,1,0,False
2,rodata_test,include/linux/rodata_test.h,1,0,False
3,rcu_barrier,rcu_barrier,1,35,True
4,mark_rodata_ro,mark_rodata_ro,0,1,True
5,printk,printk,0,5936,True
6,kernel_init_freeable,init/main.c,0,1,False
7,async_synchronize_full,async_synchronize_full,0,17,True
8,ftrace_free_init_mem,ftrace_free_init_mem,0,1,True
9,free_initmem,free_initmem,0,1,True


In [6]:
full_inlined_df = df[(df.caller_inline > 0) & (df.caller_func == 0)]
print(len(full_inlined_df) / len(df))

0.34888704025813605


In [7]:
df.caller_inline.sum() / (df.caller_inline.sum() + df.caller_func.sum())

0.6967692554561682

In [8]:
partial_inlined_df = df[(df.caller_inline > 0) & (df.caller_func > 0)]


print(len(partial_inlined_df))


partial_inlined_df[:10]

4118


,name,location,caller_inline,caller_func,external
3,rcu_barrier,rcu_barrier,1,35,True
19,kstrtobool,kstrtobool,1,47,True
25,do_raw_spin_lock,include/linux/spinlock.h,3342,204,False
30,__cmpxchg_case_acq_32,arch/arm64/include/asm/cmpxchg.h,5542,10,False
31,system_uses_lse_atomics,arch/arm64/include/asm/lse.h,12096,91,False
36,arch_local_irq_save,arch/arm64/include/asm/irqflags.h,4643,110,False
37,arch_local_save_flags,arch/arm64/include/asm/irqflags.h,4779,26,False
38,arch_irqs_disabled_flags,arch/arm64/include/asm/irqflags.h,4771,14,False
39,arch_local_irq_disable,arch/arm64/include/asm/irqflags.h,5386,140,False
45,arch_local_irq_restore,arch/arm64/include/asm/irqflags.h,4824,254,False
